In [12]:
# Dependencies

# Data manipulation libraries
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Technical indicator library
import talib as ta

# Data import library
import yfinance as yf

# Data visualisation
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Misc
from datetime import datetime as dt

In [2]:
df = yf.download('TSLA' ,period = '1d', interval = '1m')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-01-31 09:30:00-05:00,872.710022,874.000000,866.000000,868.184998,868.184998,1340711
2022-01-31 09:31:00-05:00,868.630005,870.000000,864.280029,864.683411,864.683411,227625
2022-01-31 09:32:00-05:00,864.280029,866.010010,862.049988,864.309998,864.309998,155849
2022-01-31 09:33:00-05:00,864.278381,876.450012,864.278381,875.229980,875.229980,230338
2022-01-31 09:34:00-05:00,876.429993,879.729980,875.750122,878.494995,878.494995,260670
...,...,...,...,...,...,...
2022-01-31 15:56:00-05:00,931.955017,933.669983,931.340027,933.451172,933.451172,86825
2022-01-31 15:57:00-05:00,933.510010,934.809998,932.827820,934.609985,934.609985,130266
2022-01-31 15:58:00-05:00,934.799927,935.880005,933.559998,935.255005,935.255005,181999


In [10]:
# Set up traces
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                    open=df['Open'],
                                    high=df['High'],
                                    low=df['Low'],
                                    close=df['Close'])])

# add titles
fig.update_layout(
    title = 'Tesla Price',
    yaxis_title = 'Stock Price (USD per Share)')

# X Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="30m", step="minute", stepmode="backward"),
            dict(count=6, label="90m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(step="all")
        ])
    )
)

# Display graph
fig.show()

In [6]:
# drop rows with 0 trading volume
df = df.drop(df[df['Volume'] == 0].index)

In [11]:
# create RSI column
df['RSI'] = ta.RSI(np.array(df['Close'].shift(1)), timeperiod=10)
df

,Open,High,Low,Close,Adj Close,Volume,RSI
Datetime,,,,,,,
2022-01-31 09:30:00-05:00,872.710022,874.000000,866.000000,868.184998,868.184998,1340711,NaN
2022-01-31 09:31:00-05:00,868.630005,870.000000,864.280029,864.683411,864.683411,227625,NaN
2022-01-31 09:32:00-05:00,864.280029,866.010010,862.049988,864.309998,864.309998,155849,NaN
2022-01-31 09:33:00-05:00,864.278381,876.450012,864.278381,875.229980,875.229980,230338,NaN
2022-01-31 09:34:00-05:00,876.429993,879.729980,875.750122,878.494995,878.494995,260670,NaN
...,...,...,...,...,...,...,...
2022-01-31 15:55:00-05:00,933.739990,934.580017,931.490112,931.989990,931.989990,129476,58.310537
2022-01-31 15:56:00-05:00,931.955017,933.669983,931.340027,933.451172,933.451172,86825,52.032157
2022-01-31 15:57:00-05:00,933.510010,934.809998,932.827820,934.609985,934.609985,130266,56.365991


In [15]:
# add RSI to plotly figure
fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3])

fig.add_trace(go.Candlestick(x=df.index,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'], name = 'market_data'),
             row=1, col=1)

fig.update_xaxes(
    rangeslider_visible=False)

fig.add_trace(go.Scatter(x=df.index,
                    y=df['RSI'], name = 'RSI', line=dict(color='royalblue', width=1.2)),
             row=2, col=1)

fig.show()